In [15]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [16]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [17]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [18]:
W = 250
hours = 24

insample_scenarios = Dict()
out_of_sample_scenarios = Dict()
seeds = [123,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]

for seed in seeds
    Random.seed!(seed)
    selected_scenarios = rand(1:n_scenarios, W)
    scenarios = Dict()
    counter = 1
    for i in selected_scenarios
        scenarios[counter] = all_scenarios[i]
        counter += 1
    end
    insample_scenarios[seed] = scenarios

    # create the list of index of scenarios out of sample
    out_of_sample = []
    for i in 1:n_scenarios
        if i ∉ selected_scenarios
            push!(out_of_sample, i)
        end
    end

    unselected_scenarios = Dict()
    counter = 1
    for i in out_of_sample
        unselected_scenarios[counter] = all_scenarios[i]
        counter += 1
    end
    out_of_sample_scenarios[seed] = unselected_scenarios
end

alpha = 0.9
beta = 0.1
objectiv_values = []
exp_profits = []
profits_in = []
CVaR_values = []


Any[]

In [19]:
expected_profit_list = []
#create df with seeds as column and add the p_DA_values for each seed and definec the columns
bids_df = DataFrame()

for seed in seeds
    scenarios = insample_scenarios[seed]

    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)
    unregister(model, :p_DA)

    # Define the decision variables for hour
    @variable(model, p_DA[1:hours])
    @variable(model, delta[1:W,1:hours])
    #@variable(model, delta_up[1:W,1:hours])
    #@variable(model, delta_down[1:W,1:hours])
    @variable(model, zeta)
    @variable(model, eta[1:W] >= 0)

    # Define the objective function
    @objective(model, Max, (1-beta) * sum(1/W *
    (scenarios[i][hour,"price"] * p_DA[hour]
    + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)
    + beta  * (zeta - (1/(1-alpha)) * sum(1/W * eta[i] for i in 1:W)))

    # Define the constraints
    @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
    @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
    #@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
    #@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
    #@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
    #@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
    #@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)

    @constraint(model, [i in 1:W], eta[i] >= 0)
    @constraint(model, [i in 1:W], -1 * sum((scenarios[i][hour,"price"] * p_DA[hour]
    + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours) + zeta - eta[i] <= 0)



    # Solve the optimization problem
    optimize!(model)

    # save the p_DA values
    p_DA_values = value.(p_DA)

    exp_profit = sum(1/W .*
    (scenarios[i][hour,"price"] * value.(p_DA[hour])
    + scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)


    #println(model)

    # Print the termination status
    status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Expected profit ", exp_profit)
        println("p_DA: ", value.(p_DA))
        #add the p_DA values to the df
        bids_df[!, string(seed)] = value.(p_DA)
        push!(expected_profit_list, exp_profit)
        
    else
        println("No optimal solution found")
    end
end

Expected profit 151705.65829751402
p_DA: [200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, -2.2737367544323206e-13, 200.0, 0.0, 0.0, 0.0, 200.0]
Expected profit 154063.64361532073
p_DA: [0.0, 0.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 0.0, 200.0]
Expected profit 149907.07158734958
p_DA: [0.0, 0.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0]
Expected profit 157437.83042198804
p_DA: [200.0, 0.0, 0.0, 0.0, 200.0, 197.05229261473679, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0]
Expected profit 144440.52291332762
p_DA: [0.0, 0.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 56.08322919962296, 200.0, 200.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0]
Expected profit 15584

In [20]:

W_new = n_scenarios - W
hours = 24
profits_list = []

for seed in seeds
    unselected_scenarios = out_of_sample_scenarios[seed]
    p_DA_values = value.(bids_df[!, string(seed)])
    # calculate the expected profit for the out of sample scenarios
    exp_profit = 0
    for i in 1:W_new
        profit_day = 0
        for hour in 1:hours
            profit = 0
            profit += unselected_scenarios[i][hour,"price"] * p_DA_values[hour] # should the day ahead price be fixed to the first scenario? So instead of i, 1?
            profit += unselected_scenarios[i][hour,"price"] * (unselected_scenarios[i][hour,"wind power"]-p_DA_values[hour])*((0.9.*unselected_scenarios[i][hour,"grid_excess"]) + 1.2.*(1-unselected_scenarios[i][hour,"grid_excess"]))
            profit_day += profit
        end
        #push!(profits_out, profit_day)
        exp_profit += profit_day
    end
    exp_profit = exp_profit / W_new
    push!(profits_list, exp_profit)
end

profits_list

24-element Vector{Any}:
 154973.35406540532
 152818.98503445022
 154847.13488706914
 152656.1832165012
 155001.67367857654
 153198.14695217749
 153220.76024095135
 154282.024931861
 154436.86554868685
 153495.29826739506
      ⋮
 152264.29609966144
 156804.03738275438
 154833.6035857783
 151207.3991161477
 154594.29987756055
 154226.97388208003
 153042.6669168138
 152426.25746918176
 154941.54290154448

In [22]:
#make dataframe with the expected profits, profits_list
df = DataFrame(seed = seeds, expected_profit = expected_profit_list, profit_out = profits_list)
#make a relative column
df[!, "relative"] = df[!,"profit_out"] ./ df[!,"expected_profit"].-1
df

Row,seed,expected_profit,profit_out,relative
,Int64,Any,Any,Float64
1,123,1.51706e5,1.54973e5,0.0215397
2,2,1.54064e5,152819.0,-0.00807886
3,3,1.49907e5,1.54847e5,0.0329542
4,4,1.57438e5,1.52656e5,-0.0303717
5,5,1.44441e5,1.55002e5,0.0731176
6,6,1.55845e5,1.53198e5,-0.0169815
7,7,1.51636e5,1.53221e5,0.0104498
8,8,1.51072e5,154282.0,0.0212472
9,9,146345.0,1.54437e5,0.0552933
